In [ ]:
from nustar_gen import wrappers, info, utils
import os

# NOTE!!!!
#### The code below is an *example* only for how to process straylight data for analysis.

### IT WILL NOT WORK WITH THE EXAMPLE DATA IN THIS REPOSITORY

#### Please use it as a template only for your own analysis.


### How to make images in DET1 coordinates

In [ ]:
# top == some path to your top level data directory. 
here = os.getcwd()+'/'

obs = info.Observation(seqid='10202005004', path=here, out_path='./10202005004/products/')

# Below spawns an Xselect instance behind the scenes.
# This produces an image in DET1 coordinates, which looks like the image below when you open it in ds9

evfA = obs.evtfiles['A'][0]
det1A_file = wrappers.make_det1_image(evfA,elow=3, ehigh=20)

### How to make lightcurves from DET1 data
1. Go to the DET1 image produced above and make a region that covers the straylight that you want to analyze. Note that this can be a large circle with lots of area outside of the FoV. i.e.:

<img src="./example_data/det1_image_example.png" alt="Drawing" style="width: 800px;"/>

2. Provide the full path to the region file. **Make sure to save the file using IMAGE coordinates.** See the example image above.

3. Extract the events using Xselect:

In [ ]:
# Go make the region file using ds9.
# Below spawns another XSELECT run behind the scenes to apply the region filtering in DET1 coordinates

reg_file = obs.path+'/'+obs.seqid+'/event_cl/srcB.reg'
filt_file = wrappers.extract_det1_events(obs.evtfiles['B'][0], regfile=reg_file)

# filt_file is now the full path to the extracted event file. This is located in obs.evdir by default.

4. Make a script to use nuproducts to produce a lightcurve. Note that we have turned off all of the nuproducts flags that cause it to apply the PSF, exposure, and vignetting corrections. This does do the livetime corrections. For straylight sources the motion of the telescope pointing/mast shouldn't affect the response of the instrument, and we'll assume that we'll take care of computing the effective area when doing spectroscopy. **Note:** this wrapper has a "barycorr" option, but it is currently disabled. If you're doing pulsar timing with stray light then you'll probably want to construct the nuproducts call yourself.

In [ ]:
from astropy import units as u
time_bin = 100*u.s
lc_script = wrappers.make_det1_lightcurve(filt_file, mod='B', elow=3, ehigh=10, time_bin=time_bin, obs=obs)
# lc_script is now the path to the nuproducts script to produce a lightcurve, which is stored in obs.out_path.
#
# Go run ths in the shell

5. Make a script to use nuproducts to produce a spectrum. Right now, this does not do anything with the background and does not produce an ARF on its own (see below). It will produce an RMF that will automatically be loaded when you load the data into Xspec.

In [ ]:
det1spec_script = wrappers.make_det1_spectra(filt_file, 'B', obs=obs)
# lc_script is now the path to the nuproducts script to produce a spectrum, which is stored in obs.out_path.
#
# Go run ths in the shell

6. Make and ARF. This is done "by hand" for now. To do this you also need to make a DET1 exposure map first. The detector area that you're using is currently bookkept into the ARF itself (but maybe should move to the AREASCAL keyword in the PHA file). This script uses the number of observed counts on each detectors to load in the DETABS values from the CALDB and multiplies this onto a base ARF that describes the attenuationin the Be window above the detectors. The ARF needs to be loaded by hand when using Xspec, or you can go use the fmodhead FTOOL to adjust the ANCRFILE keyword

In [ ]:
expo_script = wrappers.make_exposure_map(obs, mod, det_expo=True)

# expo_script is now the path to the nuexpomap script, which is stored in obs.out_path.
#
# If det_expo=False then this produces a Sky exposure map rather than the DET1 exposure map.
#
# Go run this in the shell

In [ ]:
# You need to specify the location of the exposure map file produce by the script above:

det1expo = f'{obs.out_path}/nu{obs.seqid}{mod}_det1_expo.fits'
filt_ev = obs.evdir+f'/nu{obs.seqid}{mod}01_cl_src{mod}.evt'    
reg_file = obs.evdir+f'/src{mod}.reg'    
utils.make_straylight_arf(det1expo, reg_file, filt_ev, mod, obs=obs)

# This produces an ARF in obs.out_path

# We're now ready for Xspec analysis, but you will need to load the ARF by hand after loading in the data.

In [ ]:
# For convenience, in the shell you can go to obs.out_path and do something like the following using the fthedit FTOOL in HEASoft:

fthedit nu10202005004B01_cl_srcB_sr.pha[1] keyword=ANCRFILE operation=add value=nu10202005004B01_cl_srcB.arf 

# ... note that the [1] is required to hit the correct FITS extension in the PHA file